# Assignment 2


#### 1. 复习上课内容以及复现课程代码

在本部分，你需要在复习上课内容和课程代码后，复现课程代码。 

#### 2. 回答以下理论题目

###       < 评阅点 >： 答案是否正确完整

###### 2.1 What conditions are required to make the BFS return the optimal solution ?

Every path has same cost and not negative or cost is nondecreasing of the depth.

##### 2.2 Is there a way to make DFS find the optimal solution ? (You may need to read some material about iterative DFS)

Assume all paths have the same cost, if we know the depth of the optimal solution, we can set DFS with a depth parameter. Or we can use iterative DFS to set depth from 1 to n and try run DFS with every possibe depth param until it find the first solution which is the optimal solution.

##### 2.3 In what conditions BFS is a better choice than DFS and vice versa ?

If we want to find an optimal solution given the cost of all paths are the same.

##### 2.4 When can we use machine learning ?

When we have a large amount of data and big computation ability and we want to find some information from data.

##### 2.5 What is the gradient of a function ?

Slope of a function, take derivative of the function by x.

##### 2.6 How can we find the maximum value of a function using the information of gradient ?

Mathematically, we can set gradient to zero and solve the function. In machine learning, we always update the parameter by adding the product of learning rate and the gradient.

#### 3. 实践部分  寻找地铁路线

### < 评阅点 >  1: 爬虫爬取数据是否完整;  2:搜索算法是否正确

In this part, although we recommend you to use Beijing subway, you still can use the subway map of any cities that you are interested in. 

Please using the search policy to implement an agent. This agent receives two input, one is @param start station and the other is @param destination. Your agent should give the optimal route based on Beijing Subway system.

Deadline: 2019-May

#### Procedures

#### 1. Get data from web.  

Some tips: 

a. You might need this package: requests[https://2.python-requests.org/en/master/] to get webpages

b.You might need to use Regular Expression and Beautiful Soap package to parse the webpages

In [4]:
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict

In [5]:
def find_all_subways(url):
    """
    :param url: str, the url which could be used to find all subway urls
    :return: lst, all urls of every subway in Beijing
    """
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        pattern = re.compile('》》<a href=\S+')
        raw_tags = pattern.findall(str(soup))
        subway_urls = []
        for tag in raw_tags:
            url = tag.split('=')[-1].strip('\"')
            subway_urls.append(url)
        return subway_urls
    else:
        print('Error Occurs...')

In [7]:
def find_stops(line_url):
    """
    :param line_url: str, the url of the subway line
    :return: dict(tuple), all stops passed by from the line
    """
    response = requests.get(line_url)
    response.encoding = 'UTF-8'
    # print(response.encoding)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # print(soup.original_encoding)
        stop_info = {}
        stops = soup.find('tbody').find_all('tr')
        for stop in stops:
            name = stop.th.text.strip()
            tds = stop.find_all('td')
            distance = int(tds[0].text)
            direction = tds[1].text
            stop_info[name] = (distance, direction)
        return stop_info
    else:
        print('Error Occurs...')

##### 2. Preprocessing data

Some tips:

a. Find a suitable way to save the data you get from the web. (Note: The way you use to save the data should be able to be used to create the graph that your agent is going to explore)

In [9]:
def build_connections(all_stops):
    """
    :param all_stops: lst of dict, stop connection of each line in Beijing
    :return: dict(list) connections between every stops
    """
    stops_connections = defaultdict(list)
    for line in all_stops:
        stops = line.keys()
        for stop in stops:
            previous, next = stop.split('——')
            way = line[stop][1].split('/')
            if len(way) == 2:
                stops_connections[previous].append(next)
                stops_connections[next].append(previous)
            elif(len(way) == 1):
                stops_connections[previous].append(next)
    return stops_connections

#### 3. Build the search agent

Build the search agent based ont he graph you built.

for example, if you use Beijing subway graoh, and you run:

\>>> search("奥体中心“，”天安门“）

You should get the result as follows: 奥体中心 -> A ->B ->C ... -> 天安门

In [12]:
def search_1(graph, start, destination):
    pathes = [[start]]  
    visited = set() 

    while pathes:
        path = pathes.pop(0) 
        froniter = path[-1] 

        if froniter in visited: continue

        successsors = graph[froniter]

        for city in successsors:  
            if city in path: continue 

            new_path = path + [city]
            pathes.append(new_path) 

            if city == destination: 
                return new_path
        visited.add(froniter)

In [15]:
SEARCH_URL = 'http://bj.bendibao.com/traffic/20141217/174565.shtm'
subways = find_all_subways(SEARCH_URL)

In [19]:
connections = [find_stops(subway) for subway in subways]
connections

[{'东直门——三元桥': (3022, '上行/下行'),
  '三元桥——T3航站楼': (18322, '下行'),
  'T3航站楼——T2航站楼': (7243, '下行'),
  'T2航站楼——三元桥': (20738, '上行')},
 {'郭公庄——大葆台': (1405, '上行/下行'),
  '大葆台——稻田': (6466, '上行/下行'),
  '稻田——长阳': (4041, '上行/下行'),
  '长阳——篱笆房': (2150, '上行/下行'),
  '篱笆房——广阳城': (1474, '上行/下行'),
  '广阳城——良乡大学城北': (2003, '上行/下行'),
  '良乡大学城北——良乡大学城': (1188, '上行/下行'),
  '良乡大学城——良乡大学城西': (1738, '上行/下行'),
  '良乡大学城西——良乡南关': (1332, '上行/下行'),
  '良乡南关——苏庄': (1330, '上行/下行')},
 {'公益西桥——新宫': (2798, '上行/下行'),
  '新宫——西红门': (5102, '上行/下行'),
  '西红门——高米店北': (1810, '上行/下行'),
  '高米店北——高米店南': (1128, '上行/下行'),
  '高米店南——枣园': (1096, '上行/下行'),
  '枣园——清源路': (1200, '上行/下行'),
  '清源路——黄村西大街': (1214, '上行/下行'),
  '黄村西大街——黄村火车站': (987, '上行/下行'),
  '黄村火车站——义和庄': (2035, '上行/下行'),
  '义和庄——生物医药基地': (2918, '上行/下行'),
  '生物医药基地——天宫院': (1811, '上行/下行')},
 {'宋家庄——肖村': (2631, '上行/下行'),
  '肖村——小红门': (1275, '上行/下行'),
  '小红门——旧宫': (2366, '上行/下行'),
  '旧宫——亦庄桥': (1982, '上行/下行'),
  '亦庄桥——亦庄文化园': (993, '上行/下行'),
  '亦庄文化园——万源街': (1728, '上行/下行'),
  '万源街——荣

In [21]:
stops_connections = build_connections(connections)
stops_connections

defaultdict(list,
            {'东直门': ['三元桥', '柳芳', '东四十条', '雍和宫'],
             '三元桥': ['东直门', 'T3航站楼', '太阳宫', '亮马桥'],
             'T3航站楼': ['T2航站楼'],
             'T2航站楼': ['三元桥'],
             '郭公庄': ['大葆台', '丰台科技园'],
             '大葆台': ['郭公庄', '稻田'],
             '稻田': ['大葆台', '长阳'],
             '长阳': ['稻田', '篱笆房'],
             '篱笆房': ['长阳', '广阳城'],
             '广阳城': ['篱笆房', '良乡大学城北'],
             '良乡大学城北': ['广阳城', '良乡大学城'],
             '良乡大学城': ['良乡大学城北', '良乡大学城西'],
             '良乡大学城西': ['良乡大学城', '良乡南关'],
             '良乡南关': ['良乡大学城西', '苏庄'],
             '苏庄': ['良乡南关'],
             '公益西桥': ['新宫', '角门西'],
             '新宫': ['公益西桥', '西红门'],
             '西红门': ['新宫', '高米店北'],
             '高米店北': ['西红门', '高米店南'],
             '高米店南': ['高米店北', '枣园'],
             '枣园': ['高米店南', '清源路'],
             '清源路': ['枣园', '黄村西大街'],
             '黄村西大街': ['清源路', '黄村火车站'],
             '黄村火车站': ['黄村西大街', '义和庄'],
             '义和庄': ['黄村火车站', '生物医药基地'],
             '生物医药基地': ['义和庄'

In [22]:
search_1(stops_connections, '北海北', '万寿路')

['北海北', '平安里', '车公庄', '车公庄西', '白石桥南', '白堆子', '军事博物馆', '公主坟', '万寿路']

### (Optional) Improve your agent to make it able to find a path based on different strategies

###  <评阅点> : 是否正确得到不同目标下的路径。

Some ideas you might want to try:

a. Find the shortest path between two stations.

b. Find the path that requires minimum transfers between two stations.

c. Combine the previous two ideas, find a more suitable path.

Compare your results with results obtained by using some apps such as Baidu map, A map, Google map or Apple map. If there is difference, try to explanate it.

## Congratulations ! You have finished the assignment of week 2.

![title](img/agent.png)

### If you have any suggestions regarding the teaching, please feel free to send them to my eamil (eric.lee.xiao@gmail.com) 